In [ ]:
import os
import psycopg2
import time
import logging

sql_dir = 'DIRECTORY LOCATION OF SQL FILES'
sql_files = [
    'LIST OF SQL FILENAMES TO EXECUTE'
]
pre_commands = []

prod_db = 'SOURCE SCHEMAS'
dev_db = 'TARGET SCHEMA'

#Credentials should be set up in b
rs_creds = {
    'database': os.environ.get('RS_DATABASE'),
    'user': os.environ.get('RS_USER'),
    'host': os.environ.get('RS_HOST'),
    'port': os.environ.get('RS_PORT'),
    'password': os.environ.get('RS_PASSWORD'),
}
db = psycopg2.connect(**(rs_creds))
db.set_session(readonly=False, autocommit=True)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

for pre_command in pre_commands:
    logger.info(' executing "%s"...' % pre_command)
    st = time.time()
    cursor = db.cursor()
    try:
        cursor.execute(pre_command)
    except:
        logger.info('     ...error raised in %.2f sec' % (time.time() - st))        
        raise
    cursor.close()
    logger.info('     ...finished in %.2f sec' % (time.time() - st)) 
for sql_file in sql_files:
    with open(os.path.join(sql_dir, sql_file + '.sql'), 'r') as infile:
        query = infile.read()
    query = query.replace('set search_path to %s' % prod_db, 'set search_path to %s' % dev_db)
    logger.info(' executing %s...' % sql_file)
    st = time.time()
    cursor = db.cursor()
    try:
        cursor.execute(query)
    except:
        logger.info('     ...error raised in %.2f sec' % (time.time() - st))
        logger.info(query)
        raise
    cursor.close()
    logger.info('     ...finished in %.2f sec' % (time.time() - st))